## Install Package

In [1]:
!pip install ultralytics
!pip install roboflow                

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Load & Set Dataset

In [1]:
from roboflow import Roboflow
from ultralytics import YOLO, settings

rf = Roboflow(api_key="A0nVf3Qxv9Ze7RrtqV3W")
project = rf.workspace("landslide-28gdu").project("landslide-area-segmentation")
version = project.version(6)
dataset = version.download("yolov11")

/Users/garrygeng/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


loading Roboflow workspace...
loading Roboflow project...


In [2]:
settings.update({"datasets_dir": "/Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6"})
print(settings)

JSONDict("/Users/garrygeng/Library/Application Support/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "b406ac71b2beb8fa80656ee616450baa29492ac3e2b490aec1ee2ab2e2a1d082",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}


## Test for the original model

In [4]:
model = YOLO("yolo11l.pt")
# Perform object detection on an image
results = model("Landslide-Area-Segmentation-6/test/images/12_flip_jpg.rf.19afbd5f6be57496c3745198e9b41755.jpg")

# Visualize the results
for result in results:
    result.show()



image 1/1 /Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6/test/images/12_flip_jpg.rf.19afbd5f6be57496c3745198e9b41755.jpg: 640x640 (no detections), 379.4ms
Speed: 2.4ms preprocess, 379.4ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 640)


## Training the YOLO 11 N model

In [ ]:
import torch

torch.manual_seed(42)

dataset_config = './Landslide-Area-Segmentation-6/data.yaml'
model = YOLO("yolo11l.pt")

results = model.train(
    data=dataset_config,
    epochs=10,
    imgsz=640,
    batch=16,
    name='landslide_detection',
    project='runs/detect',
    # device='mps',
)

# 评估模型
metrics = model.val()

# 保存最佳模型权重
best_model_weights = 'runs/detect/landslide_detection/weights/best.pt'
print(f"Best model weights saved to {best_model_weights}")

Ultralytics 8.3.74 🚀 Python-3.9.6 torch-2.0.1 MPS (Apple M2)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=./Landslide-Area-Segmentation-6/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=runs/detect, name=landslide_detection15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning /Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6/train/labels... 1623 images, 3 backgrounds, 0 corrupt: 100%|██████████| 1623/1623 [00:00<00:00, 3455.35it/s]

train: WARNING ⚠️ /Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6/train/images/13_translate_images_jpg.rf.ababd1c7422044f02d5ea39d1f6b2843.jpg: 1 duplicate labels removed
train: New cache created: /Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6/train/labels.cache



val: Scanning /Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6/valid/labels... 130 images, 0 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<00:00, 3373.73it/s]

val: New cache created: /Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6/valid/labels.cache


Plotting labels to runs/detect/landslide_detection15/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/landslide_detection15
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/102 [00:00<?, ?it/s]

: 

## Val

In [14]:
# 加载训练好的模型
model = YOLO('yolo11l.pt')
model = YOLO('./weights/best.pt')
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

Ultralytics 8.3.74 🚀 Python-3.9.6 torch-2.0.1 CPU (Apple M2)
YOLO11l summary (fused): 464 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning /Users/garrygeng/Desktop/YOLO/Landslide-Area-Segmentation-6/valid/labels.cache... 130 images, 0 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [02:23<00:00, 15.94s/it]


                   all        130        209      0.722       0.47      0.562      0.389
             landslide        130        195      0.754      0.631      0.646      0.425
                object          2          2      0.413        0.5      0.496      0.347
                 rocks          1         12          1       0.28      0.544      0.394
Speed: 0.9ms preprocess, 1098.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val8


array([    0.42463,     0.34739,     0.39395])

## Predict

In [15]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11l.pt")  # pretrained YOLO11n model
model = YOLO('./weights/best.pt')

# Run batched inference on a list of images
results = model(["random-shapes-to-random-images-v0-8nayrq3y25oa1.webp", "random-images-v0-gegqtubaln0b1.webp"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk


0: 640x640 (no detections), 401.2ms
1: 640x640 2 landslides, 401.2ms
Speed: 2.0ms preprocess, 401.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
